In [1]:

import pandas as pd

In [2]:
def signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    Pre_open = df.Open.iloc[-2]
    Pre_close = df.Close.iloc[-2]

    #Bearish Signal
    if (open > close and Pre_close>Pre_open and
        close < Pre_open and Pre_close<=open):
        return 1
    
    #Bullish signal
    elif (
        close>open and Pre_close<Pre_open and
        Pre_open<close and Pre_close>= open):
        return 2
    
    #no signal
    else:
        return 0


In [3]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from datetime import datetime

API_KEY = 'your_api_key'
SECRET_KEY = 'your_secret_key'

In [25]:
def get_candle(n):
    client = TradingClient(API_KEY, SECRET_KEY)
    data_client = StockHistoricalDataClient(API_KEY, SECRET_KEY)
    request_para = StockBarsRequest(
        symbol_or_symbols='AAPL',
        timeframe=TimeFrame.Minute ,
        start=datetime(2025, 1,1),
        end=datetime(2025, 7,1)
    )

    bars = data_client.get_stock_bars(request_para)
    df = bars.df.reset_index()
    df = df.drop('symbol', axis=1)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.set_index('timestamp')
    candles = df.resample('15T').agg({
        'open': 'first',
        'high' : 'max',
        'low':'min',
        'close':'last',
        'volume':'sum',
    }).dropna()
    return candles.sort_values(by='timestamp', ascending=False).head(n)
candle = get_candle(3)
candle

C:\Users\AMIT\AppData\Local\Temp\ipykernel_3832\1673101615.py:16: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  candles = df.resample('15T').agg({


,open,high,low,close,volume
timestamp,,,,,
2025-06-30 23:45:00+00:00,205.80,205.856,205.44,205.4400,42801.0
2025-06-30 23:30:00+00:00,205.50,205.960,205.50,205.8800,72721.0
2025-06-30 23:15:00+00:00,205.42,205.600,205.28,205.5467,32861.0


In [42]:
def trading_job():
    candle = get_candle(3)

    new_df = pd.DataFrame({
        'Open' : candle['open'][:3],
        'Close' : candle['close'][:3],
        'High' : candle['high'][:3],
        'Low' : candle['low'][:3],
    })

    new_df['Open'] = new_df['Open'].astype(float)
    new_df['Close'] = new_df['Close'].astype(float)
    new_df['High'] = new_df['High'].astype(float)
    new_df['Low'] = new_df['Low'].astype(float)

    signal = signal_generator(new_df.iloc[1:,:])
    
    pnl_Ratio = 2
    pre_candleR = abs(new_df['Open'].iloc[1]- new_df['Close'].iloc[1])

    SLbuy = new_df['Open'].iloc[0] - pre_candleR
    SLsell = new_df['Open'].iloc[0] + pre_candleR

    TPbuy = new_df['Open'].iloc[0] + pre_candleR*pnl_Ratio
    TPsell = new_df['Open'].iloc[0] - pre_candleR*pnl_Ratio

    print(new_df[:-1,:])
    print(TPbuy, " ", SLbuy, " ", TPsell, " ", SLsell)
trading_job()




C:\Users\AMIT\AppData\Local\Temp\ipykernel_3832\1673101615.py:16: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  candles = df.resample('15T').agg({


InvalidIndexError: (slice(None, -1, None), slice(None, None, None))